Librerias

In [9]:
# Copyright (c) 2024 Juan Diego Díaz Domínguez
# Todos los derechos reservados.

In [10]:
import pdfplumber
import os
import json
import spacy
from spacy.util import minibatch, compounding
import random
import json
from sklearn.model_selection import train_test_split
from spacy.util import minibatch, compounding
import random
import warnings
from spacy.training import Example
import pandas as pd
import base64
import openai
import openpyxl


# Suprimir todos los warnings
warnings.filterwarnings("ignore")

<b>Extraer el texto de las diferentes facturas</b>

In [ ]:
ruta_directorio = "directorio con las facturas"
openai.api_key = "api de OpenAI"

for archivo in os.listdir(ruta_directorio):
    if archivo.endswith(".pdf"):
        ruta_pdf = os.path.join(ruta_directorio, archivo)
        with pdfplumber.open(ruta_pdf) as pdf:
            texto = ""
            for pagina in pdf.pages:
                texto += pagina.extract_text()
        nombre_archivo_txt = archivo.replace(".pdf", ".txt")
        ruta_txt = os.path.join(ruta_directorio, nombre_archivo_txt)
        with open(ruta_txt, "w", encoding="utf-8") as archivo_txt:
            archivo_txt.write(texto)
            print(f"Texto extraído guardado en: {ruta_txt}")

In [ ]:
facturas_json = []
for archivo in os.listdir(ruta_directorio):
    if archivo.endswith(".txt"):
        ruta_txt = os.path.join(ruta_directorio, archivo)
        
        # Leer el contenido del archivo .txt
        with open(ruta_txt, "r", encoding="utf-8") as archivo_txt:
            texto = archivo_txt.read()
        
        # Añadir el texto al formato JSON
        factura = {
            "data": {
                "text": texto
            }
        }
        facturas_json.append(factura)

        # Imprimir el nombre del archivo que se está procesando
        print(f"Procesando archivo: {archivo}")

ruta_json = os.path.join(ruta_directorio, "facturas_import.json")
with open(ruta_json, "w", encoding="utf-8") as archivo_json:
    json.dump(facturas_json, archivo_json, ensure_ascii=False, indent=4)

print(f"Archivo JSON creado: {ruta_json}")

<b>En este punto, se procede al etiquetado manual de los diferentes campos con la herramienta Label Studio</b>

In [ ]:
ruta_json = os.path.join(ruta_directorio, "DatosEtiquetados.json")
with open(ruta_json, "r", encoding="utf-8") as f:
    datos_etiquetados = json.load(f)

# Dividir los datos en train y test
train_data, test_data = train_test_split(datos_etiquetados, test_size=0.2, random_state=42)

with open("facturas_train_data.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, ensure_ascii=False, indent=4)

with open("facturas_test_data.json", "w", encoding="utf-8") as f:
    json.dump(test_data, f, ensure_ascii=False, indent=4)

print("Datos divididos en entrenamiento y prueba.")


In [5]:
TRAIN_DATA = []

for item in datos_etiquetados:
    texto = item["data"]["text"]
    entidades = []

    # Recorrer las anotaciones
    for anotacion in item["annotations"]:
        for resultado in anotacion["result"]:
            start = resultado["value"]["start"]
            end = resultado["value"]["end"]
            label = resultado["value"]["labels"][0]
            entidades.append((start, end, label))
    TRAIN_DATA.append((texto, {"entities": entidades}))

# Guardar los datos de entrenamiento
ruta_json = os.path.join(ruta_directorio, "facturas_train_data_spacy.json")
with open(ruta_json, "w", encoding="utf-8") as f:
    json.dump(TRAIN_DATA, f, ensure_ascii=False, indent=4)

print("Datos de entrenamiento preparados para spaCy.")

Datos de entrenamiento preparados para spaCy.


In [ ]:
ruta_json = os.path.join(ruta_directorio, "facturas_train_data_spacy.json")
with open(ruta_json, "r", encoding="utf-8") as f:
    TRAIN_DATA = json.load(f)

train_data, test_data = train_test_split(TRAIN_DATA, test_size=0.2, random_state=42)

ruta_json = os.path.join(ruta_directorio, "facturas_train.json")
with open(ruta_json, "w", encoding="utf-8") as f:
    json.dump(train_data, f, ensure_ascii=False, indent=4)
    
ruta_json = os.path.join(ruta_directorio, "facturas_test.json")
with open(ruta_json, "w", encoding="utf-8") as f:
    json.dump(test_data, f, ensure_ascii=False, indent=4)

print("Datos divididos en entrenamiento y prueba.")


In [ ]:
ruta_directorio = "directorio de las facturas"

ruta_json = os.path.join(ruta_directorio, "facturas_train.json")
with open(ruta_json, "r", encoding="utf-8") as f:
    TRAIN_DATA = json.load(f)

ruta_json = os.path.join(ruta_directorio, "facturas_test.json")
with open(ruta_json, "r", encoding="utf-8") as f:
    TEST_DATA = json.load(f)

# Configuraciones para probar diferentes modelos, se pueden añadir o eliminar para mejorar
configuraciones = [
    {"iteraciones": 20, "dropout": 0.3},
    {"iteraciones": 30, "dropout": 0.5},
    {"iteraciones": 40, "dropout": 0.6}
]

resultados = []
mejor_modelo = None
menor_perdida = float('inf')

for config in configuraciones:
    nlp = spacy.blank("es")

    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner", last=True)

    # Añadir etiquetas
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # Entrenar
    optimizer = nlp.begin_training()

    for itn in range(config["iteraciones"]):
        random.shuffle(TRAIN_DATA)
        losses = {}
        batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            examples = []

            for text, annotations in batch:
                # Crear un objeto `Example`
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                examples.append(example)

            nlp.update(examples, drop=config["dropout"], losses=losses)
        
        print(f"Iteración {itn}, Pérdidas: {losses}")

    # Evaluar el modelo con los datos test
    y_true = []
    y_pred = []

    for text, annotations in TEST_DATA:
        doc = nlp(text)
        true_entities = {label: (start, end) for start, end, label in annotations.get("entities")}
        pred_entities = {ent.label_: (ent.start_char, ent.end_char) for ent in doc.ents}
        y_true.append(true_entities)
        y_pred.append(pred_entities)

    resultados.append({
        "configuración": config,
        "modelo": nlp,
        "perdidas_finales": losses
    })

    # Verificar si el modelo actual tiene la menor pérdida
    # y en caso de que sea mejor, guardarlo
    if losses['ner'] < menor_perdida:
        menor_perdida = losses['ner']
        mejor_modelo = nlp

for res in resultados:
    print(f"Configuración: {res['configuración']}, Pérdidas finales: {res['perdidas_finales']}")

# Guardar el mejor modelo
if mejor_modelo:
    mejor_modelo.to_disk("mejor_modelo_facturas_ner")
    print(f"Mejor modelo guardado con pérdida: {menor_perdida}")



In [ ]:
nlp = spacy.load("mejor_modelo_facturas_ner")

# Ruta del directorio donde están los PDF
ruta_directorio_pdfs = ruta_directorio  
rutas_facturas = [os.path.join(ruta_directorio_pdfs, archivo) for archivo in os.listdir(ruta_directorio_pdfs) if archivo.endswith(".pdf")]

def extraer_texto_pdf(ruta_pdf):
    with pdfplumber.open(ruta_pdf) as pdf:
        texto = ""
        for pagina in pdf.pages:
            texto += pagina.extract_text() + "\n"
    return texto

def convertir_pdf_a_base64(ruta_pdf):
    with open(ruta_pdf, "rb") as archivo_pdf:
        pdf_bytes = archivo_pdf.read()
        pdf_base64 = base64.b64encode(pdf_bytes).decode('utf-8')
    return pdf_base64

def extraer_campos_factura(modelo_nlp, texto_factura):
    doc = modelo_nlp(texto_factura)
    campos = {
        "Fecha": None,
        "CIF": None,
        "Número de Factura": None,
        "Teléfono": None,
        "Importe Total": None
    }

    for ent in doc.ents:
        if ent.label_ == "FECHA":
            campos["Fecha"] = ent.text
        elif ent.label_ == "CIF":
            campos["CIF"] = ent.text
        elif ent.label_ == "NUM_FACTURA":
            campos["Número de Factura"] = ent.text
        elif ent.label_ == "TELÉFONO":
            campos["Teléfono"] = ent.text
        elif ent.label_ == "TOTAL":
            campos["Importe Total"] = ent.text

    return campos

api_key="API key de OpenAI",
def extraer_razon_social_con_chatgpt(texto_factura):
    prompt = (
        "Eres un experto en la extracción de información relevante de facturas comerciales. "
        "Tu tarea es analizar el texto de una factura y extraer la razón social de la empresa emisora o cliente. "
        f"Texto de la factura:\n{texto_factura}\n\n"
        "Proporcióname únicamente la razón social de la empresa de manera clara y precisa."
    )
    
    respuesta = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=50,
        n=1,
        temperature=0.2
    )
    razon_social = respuesta.choices[0].message['content'].strip()
    return razon_social

datos_facturas = []

# Procesar
for ruta in rutas_facturas:
    # Extraer texto de la factura
    texto_factura = extraer_texto_pdf(ruta)

    campos_factura = extraer_campos_factura(nlp, texto_factura)

    pdf_base64 = convertir_pdf_a_base64(ruta)
    
    razon_social = extraer_razon_social_con_chatgpt(texto_factura)

    campos_factura["Texto Completo"] = texto_factura
    campos_factura["PDF Base64"] = pdf_base64
    campos_factura["Razón Social"] = razon_social

    datos_facturas.append(campos_factura)

df_facturas = pd.DataFrame(datos_facturas)

print(df_facturas)

# Guardar la tabla en un archivo CSV o Excel (Excel tiene mejor presentación)
df_facturas.to_csv("facturas_procesadas.csv", index=False)
df_facturas.to_excel("facturas_procesadas.xlsx", index=False)
